# The Birthday Attack, In Graphs

So, out of my own curiosity from a problem I saw online, trying to figure out what are the odds that people have randomly generated the same map in Minecraft, my mind went back to the birthday attack problem, because I had managed to get an answer of about 1 in 4 if all the active Minecraft players have generated about 20 maps (closer to about 23 maps for 141M players in the 64-bit space). This kind of impressed me, because knowing that the 64-bit space is about 10^19, the fact that we'd land in 1 in 4 is surprising with 1.4 * 10^8 players.  Amazingly, it was surprisingly both very high and very low to me. With birthday attack stuff, I'm used to either seeing situations be close to 1 in 10^18 (ie, basically never), or very high, like >95%.  25% is, surprisingly, weird.

So I want to graph it.  I want to see how the probability of the birthday attack looks in graphs.

## The Birthday Attack?

For those not familiar with the idea of a birthday attack, you're best off reading a couple of Wikipedia articles about it, namely the [Birthday Problem](https://en.wikipedia.org/wiki/Birthday_problem) and the [Birthday Attack](https://en.wikipedia.org/wiki/Birthday_attack).  The name stems from the idea that in a room of 23 people, the odds that at least two people will share the same birthday on the Gregorian Calendar of 365 days is greater than 50%.

You can [approximate the probability as follows](https://en.wikipedia.org/wiki/Birthday_problem#Approximations):

$
\large
\begin{align*}
    p(n, d) &\approx 1 - e^{-\frac{n(n-1)}{2d}}  \\
    &\approx 1 - e^{-\frac{n^2}{2d}}
\end{align*}
$

In code, the two approximations are written below.

In [1]:
import math

def birthday_attack_probability_apx1(n, d):
    return 1 - math.exp(-1 * ((n * (n-1))/(2 * d)))

def birthday_attack_probability_apx2(n, d):
    return 1 - math.exp((-1 * n**2)/(2 * d))


Just to test this code out, let's use the standard birthday probabilities:

In [2]:
d = 365
n = 23

print(birthday_attack_probability_apx1(n, d))
print(birthday_attack_probability_apx2(n, d))

0.5000017521827107
0.5155095380615168


So the two values differ a little bit from the more correct value of about 0.507 for 23 people in 365 using a permutations perspective, but will be good enough for large numbers, where trying to do something silly, like a value of $\large1 - \frac{_{d}P_{n}}{d^n}$ with $d = 2^{64} \approx 1.8 \times 10^{19}$ and say $n = 5.1 \times 10 ^ 9$, where the stupidly large exponents would just make the code go whack.  Just for my sake, let's see the code.

In [3]:
import math  # already done above, but doing it here for this block to work standalone.
import time

def birthday_attack_probability(n, d):
    start_time = time.time()
    ret_val = math.perm(d, n) / (d**n)
    end_time = time.time()
    execution_time = (end_time - start_time) * 1000.0
    print(f'Execution time: {execution_time:.6f} ms.')
    return 1 - ret_val

And here's the execution for the birthday probabilities.

In [4]:
d = 365
n = 23

birthday_attack_probability(n, d)

Execution time: 0.026941 ms.


0.5072972343239854

It's fast for small numbers and obviously calculates more correctly, but let's bump up each of these.

In [5]:
# 5.0 * 10^4 in 2^32 is ~25% from the wiki articles

d = 2**32
n = 50000

birthday_attack_probability(n, d)


Execution time: 3063.699007 ms.


0.25250944791713614

That's... significantly slower. Take my word that trying to do this with the 50% interval of 64 bits, so `d = 2**64` and `d = 5.06 * 10**9` is going to take... way too long.

In [6]:
d = 2**64
n = int(5.06 * 10**9)

# Don't waste your time running this.
# birthday_attack_probability(n, d)

# For the approximation:
birthday_attack_probability_apx1(n, d)

0.5004197176829943

So, for this exercise, we'll just do the approximations. Specifically, I'll use `birthday_attack_probability_apx1(n, d) going forward`.

For the reverse, the minimum number of items ($n$) needed to have a certain probability ($p$) of finding a collision in space ($H$, the equivalent of $d$ from above) is [from the derived equation](https://en.wikipedia.org/wiki/Birthday_attack#Mathematics):

$ \large n(p; H) \approx \sqrt{ 2H \ln \frac{1}{1 - p} } $

and is implemented by the following code:

In [7]:
def birthday_attack_num_required_apx(p, h):
    # we use math.ceil because the square root part will return a float.
    # logically, however, we're looking for countable numbers, so we need
    # the next highest whole number.
    return math.ceil(math.sqrt(2 * h * math.log(1 / (1 - p))))

For example, for the birthday probability of 50%

In [8]:
p = 0.5
h = 365

birthday_attack_num_required_apx(p, h)

23

And for our far larger spaces:

In [9]:
p = 0.25
h = 2**32
birthday_attack_num_required_apx(p, h)

49711

In [10]:
p = 0.5
h = 2**64
birthday_attack_num_required_apx(p, h)

5056937541

Finally, we can generate that really nice birthday attack table that exists in the wikipedia articles, using some code, plus add some extra values on the ends for some more fun.

(Note: the ~8.512 bit value is $log_{2}(365)$, i.e., the normal birthday attack.)

In [11]:
import pandas as pd
import numpy as np

# Converting this to be numpy compatible
def birthday_attack_num_required_apx(p, h):
    # we use math.ceil because the square root part will return a float.
    # logically, however, we're looking for countable numbers, so we need
    # the next highest whole number.
    return int(np.ceil(np.sqrt(2 * np.float128(h) * np.log(1 / (1 - p)))))

def sci_not(n):
    """ Generate scientific notation values for display. """
    if n < 0.001 or n >= 1000:
        return f'{n:.1e}'
    return str(n)

# Inputs as constants.
BIT_VALUES = [np.log2(365), 32, 40, 48, 64, 96, 128, 192, 256, 384, 512]
PROBABILITIES = [10**-18, 10**-15, 10**-12, 10**-9, 10**-6, 0.001, 0.01, 0.25, 0.50, 0.75, 0.9, 0.99, 0.999, 0.9999]

# Converting them to pandas series so we can multiplex them together.
BIT_VALUES_NP = pd.Series(
    [2 ** bits for bits in BIT_VALUES],
    [str(i) for i in BIT_VALUES]
)
PROBABILITIES_NP = pd.Series(
    [np.longdouble(p) for p in PROBABILITIES],
    [sci_not(p) for p in PROBABILITIES]
)

# calculate the data for the dataframe
num_required_data = {
    p_idx: [
        sci_not(birthday_attack_num_required_apx(p, h))
        for bv_idx, h in BIT_VALUES_NP.items()]
    for p_idx, p in PROBABILITIES_NP.items()
}

num_required = pd.DataFrame(
    num_required_data,
    index=BIT_VALUES_NP.index,
    columns=PROBABILITIES_NP.index
)

num_required


,1.0e-18,1.0e-15,1.0e-12,1.0e-09,1.0e-06,0.001,0.01,0.25,0.5,0.75,0.9,0.99,0.999,0.9999
8.51175265376738,1,1,1,1,1,1,3,15,23,32,41,58,72,82
32,1,1,1,3,93,2.9e+03,9.3e+03,5.0e+04,7.7e+04,1.1e+05,1.4e+05,2.0e+05,2.4e+05,2.8e+05
40,1,1,2,47,1.5e+03,4.7e+04,1.5e+05,8.0e+05,1.2e+06,1.7e+06,2.3e+06,3.2e+06,3.9e+06,4.5e+06
48,1,1,24,751,2.4e+04,7.5e+05,2.4e+06,1.3e+07,2.0e+07,2.8e+07,3.6e+07,5.1e+07,6.2e+07,7.2e+07
64,6,193,6.1e+03,1.9e+05,6.1e+06,1.9e+08,6.1e+08,3.3e+09,5.1e+09,7.2e+09,9.2e+09,1.3e+10,1.6e+10,1.8e+10
96,3.9e+05,1.3e+07,4.0e+08,1.3e+10,4.0e+11,1.3e+13,4.0e+13,2.1e+14,3.3e+14,4.7e+14,6.0e+14,8.5e+14,1.0e+15,1.2e+15
128,2.6e+10,8.2e+11,2.6e+13,8.2e+14,2.6e+16,8.3e+17,2.6e+18,1.4e+19,2.2e+19,3.1e+19,4.0e+19,5.6e+19,6.9e+19,7.9e+19
192,1.1e+20,3.5e+21,1.1e+23,3.5e+24,1.1e+26,3.5e+27,1.1e+28,6.0e+28,9.3e+28,1.3e+29,1.7e+29,2.4e+29,2.9e+29,3.4e+29
256,4.8e+29,1.5e+31,4.8e+32,1.5e+34,4.8e+35,1.5e+37,4.8e+37,2.6e+38,4.0e+38,5.7e+38,7.3e+38,1.0e+39,1.3e+39,1.5e+39
384,8.8e+48,2.8e+50,8.9e+51,2.8e+53,8.9e+54,2.8e+56,8.9e+56,4.8e+57,7.4e+57,1.0e+58,1.3e+58,1.9e+58,2.3e+58,2.7e+58


---

## The Graphs

(To be continued...)